In [70]:
import spacy
nlp = spacy.load('ro_core_news_sm')

In [71]:
alignment_dict = dict()
#Alinieri cu eflomal
def read_sentences(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file]

def parse_alignments(alignment_file, source_sentences, target_sentences):
    with open(alignment_file, 'r', encoding='utf-8') as file:
        for line_num, line in enumerate(file):
            alignment_pairs = line.strip().split()
            for pair in alignment_pairs:
                src_index, tgt_index = map(int, pair.split('-'))
                src_word = source_sentences[line_num].split()[src_index]
                tgt_word = target_sentences[line_num].split()[tgt_index]
                yield src_word, tgt_word

# Read sentences from source and target files
source_sentences = read_sentences('corpus.rup')
target_sentences = read_sentences('corpus.ro')

# Extract and print aligned word pairs
for src_word, tgt_word in parse_alignments('forward.align', source_sentences, target_sentences):
    print(f"{src_word} - {tgt_word}")
    alignment_dict[src_word] = tgt_word

LITERATURĂ - LITERATURĂ
POPULARĂ - POPULARĂ
ANECDOTE - ANECDOTE
LEGENDE - LEGENDE
BASME - BASME
LUPLU - LUPUL
CU - ȘI
VULPEA - VULPEA
Eara - fost
țe - ce
nu - n-a
ș-eara. - fost.
Eara - fost
un - un
lup, - lup,
- - -
Masturlu-Nicola, - Cumătrul-Nicola,
și - și-o
vulpe, - vulpe,
- - -
Chira-Mara. - Cumătra-Mara.
Nîși - doi
s-feațiră - s-au
soț, - întovărășit,
se-avină - vâneze
deadun - împreună,
și - iar
avinaticlu - vânatul,
s-lu-mpartă - și-au
frățește. - zis,
avinaticlu - l-or
s-lu-mpartă - împărți
frățește. - frățește.
Tu - Ieșind
ișirea - vânat,
- - dar,
aflară - peste
ună - o
oală - oală
cu - cu
ñeare. - miere.
- - -
Ha!, - Aha!,
s-aurñi - repezi
Masturlu-Nicola, - Cumătru-Nicola,
- - -
u - o
mîc! - pap!
- - -
Năinte - întâi
ș-năinte - întâi
nu - nu
prinde - trebuie
s-dzîț: - spui:
„va - „o
mîc“, - pap!“,
ma: - ci
„va - „o
mîcăm“, - păpăm“,
îl’i - i-a
fațe - zis
Chira-Mara. - Cumătra-Mara.
Ș-apoia, - Apoi,
nu - nu
dzîț - gândești,
ninga - nici
nu - n-am
și, - oală,
ai! - gata:
s-u

In [72]:
import json
import pandas as pd
from collections import defaultdict
import tqdm
excel_path = "Papahagi.xls"
df = pd.read_excel(excel_path, header = None)

df.columns = ["POS", "aro", "ro", "origine", "IDK", "autor"]
df.head()


import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

with open('corpus.rup', 'r', encoding='utf-8') as file:
    aro_text = file.read()

aro_tokens = word_tokenize(aro_text.lower())
vectorizer = CountVectorizer()

direct_translation = dict()
for word in tqdm.tqdm(aro_tokens):
    if word in df["aro"].values.tolist():
        # print(word)
        # print(df[df["aro"] == word]["ro"].values.tolist())
        direct_translation[word] = df[df["aro"] == word]["ro"].values.tolist()



[nltk_data] Downloading package punkt to /home/stefan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|███████████████████████████████████| 74343/74343 [00:42<00:00, 1756.70it/s]


In [73]:
with open("corpus.ro", "r", encoding="utf-8") as romanian_file:
    romanian_text = romanian_file.read()

with open("corpus.rup", "r", encoding="utf-8") as aromanian_file:
    aromanian_text = aromanian_file.read()

doc_ro = nlp(romanian_text)

romanian_tokens = [token.text for token in doc_ro]
romanian_pos_tags = [token.pos_ for token in doc_ro]

# Tokenize the Aromanian text
aromanian_tokens = aromanian_text.split()



In [74]:

aromanian_pos_tags = []
total_words = 0
unknown_words = 0
import tqdm


total_words = 0
unknown_words = 0


for token in tqdm.tqdm(aromanian_tokens):
    total_words += 1
    pos_tag_assigned = False

    if token in alignment_dict:
        romanian_equivalent = alignment_dict[token]
        try:
            index = romanian_tokens.index(romanian_equivalent)
            aromanian_pos_tags.append(romanian_pos_tags[index])
            pos_tag_assigned = True
        except ValueError:
            pass 

    if not pos_tag_assigned and token in direct_translation:
        romanian_direct_equivalent = direct_translation[token]
        try:
            index = romanian_tokens.index(romanian_direct_equivalent)
            aromanian_pos_tags.append(romanian_pos_tags[index])
            pos_tag_assigned = True
        except ValueError:
            pass  

    if not pos_tag_assigned:
        doc_ro_token = nlp(token)
        if len(doc_ro_token) > 0:
            aromanian_pos_tags.append(doc_ro_token[0].pos_)
        else:
            unknown_words += 1
            aromanian_pos_tags.append("UNKNOWN")


print(f"Total words processed: {total_words}")
print(f"Unknown words: {unknown_words}")



for token, pos_tag in zip(aromanian_tokens, aromanian_pos_tags):
    print(f"{token} ({pos_tag})")

100%|████████████████████████████████████| 55304/55304 [01:28<00:00, 625.20it/s]


Total words processed: 55304
Unknown words: 0
LITERATURĂ (NOUN)
POPULARĂ (ADJ)
ANECDOTE (PROPN)
LEGENDE (PROPN)
BASME (PROPN)
LUPLU (NOUN)
CU (ADP)
VULPEA (NUM)
Eara (AUX)
țe (PRON)
nu (ADV)
ș-eara. (DET)
Eara (AUX)
un (DET)
lup, (NOUN)
- (PUNCT)
Masturlu-Nicola, (PROPN)
și (CCONJ)
ună (PRON)
vulpe, (VERB)
- (PUNCT)
Chira-Mara. (PROPN)
Nîși (PROPN)
s-feațiră (AUX)
soț, (NOUN)
se-avină (AUX)
deadun (VERB)
și (CCONJ)
avinaticlu (NOUN)
s-lu-mpartă (VERB)
frățește. (AUX)
Tu (ADP)
ișirea (AUX)
avinare (NOUN)
- (PUNCT)
aflară (ADP)
ună (PRON)
oală (NOUN)
cu (ADP)
ñeare. (NOUN)
- (PUNCT)
Ha!, (PROPN)
s-aurñi (VERB)
Masturlu-Nicola, (PROPN)
- (PUNCT)
va (PRON)
u (PRON)
mîc! (NOUN)
- (PUNCT)
Năinte (ADV)
ș-năinte (ADV)
nu (ADV)
prinde (NOUN)
s-dzîț: (NOUN)
„va (PUNCT)
u (PRON)
mîc“, (NOUN)
ma: (CCONJ)
„va (PUNCT)
u (PRON)
mîcăm“, (PROPN)
îl’i (PRON)
fațe (VERB)
Chira-Mara. (PROPN)
Ș-apoia, (AUX)
nu (ADV)
dzîț (AUX)
tine, (PRON)
ninga (DET)
nu (ADV)
o-aflăm (NOUN)
- (PUNCT)
și, (CCONJ)
ai! (DET)

In [75]:
print(len(aromanian_tokens), len(aromanian_pos_tags))

55304 55304


# Hidden Markov Model cu algoritm Viterbi

In [76]:
aromanian_tokens = aromanian_tokens
aromanian_pos_tags = [x if x != '' else 'X' for x in aromanian_pos_tags]
aromanian_pos_tags = [x if x != "PROPN" else "NOUN" for x in aromanian_pos_tags]

train_tagged_words = [(aromanian_tokens[i], aromanian_pos_tags[i]) for i in range(len(aromanian_tokens))]

print(train_tagged_words[:50])

[('LITERATURĂ', 'NOUN'), ('POPULARĂ', 'ADJ'), ('ANECDOTE', 'NOUN'), ('LEGENDE', 'NOUN'), ('BASME', 'NOUN'), ('LUPLU', 'NOUN'), ('CU', 'ADP'), ('VULPEA', 'NUM'), ('Eara', 'AUX'), ('țe', 'PRON'), ('nu', 'ADV'), ('ș-eara.', 'DET'), ('Eara', 'AUX'), ('un', 'DET'), ('lup,', 'NOUN'), ('-', 'PUNCT'), ('Masturlu-Nicola,', 'NOUN'), ('și', 'CCONJ'), ('ună', 'PRON'), ('vulpe,', 'VERB'), ('-', 'PUNCT'), ('Chira-Mara.', 'NOUN'), ('Nîși', 'NOUN'), ('s-feațiră', 'AUX'), ('soț,', 'NOUN'), ('se-avină', 'AUX'), ('deadun', 'VERB'), ('și', 'CCONJ'), ('avinaticlu', 'NOUN'), ('s-lu-mpartă', 'VERB'), ('frățește.', 'AUX'), ('Tu', 'ADP'), ('ișirea', 'AUX'), ('avinare', 'NOUN'), ('-', 'PUNCT'), ('aflară', 'ADP'), ('ună', 'PRON'), ('oală', 'NOUN'), ('cu', 'ADP'), ('ñeare.', 'NOUN'), ('-', 'PUNCT'), ('Ha!,', 'NOUN'), ('s-aurñi', 'VERB'), ('Masturlu-Nicola,', 'NOUN'), ('-', 'PUNCT'), ('va', 'PRON'), ('u', 'PRON'), ('mîc!', 'NOUN'), ('-', 'PUNCT'), ('Năinte', 'ADV')]


In [77]:
tags = set(aromanian_pos_tags)
print(len(tags))
print(tags)
vocab = set(aromanian_tokens)

16
{'NUM', 'ADP', 'PART', 'NOUN', 'ADV', 'CCONJ', 'PUNCT', 'INTJ', 'SYM', 'DET', 'X', 'SCONJ', 'PRON', 'ADJ', 'VERB', 'AUX'}


In [81]:
print(len(train_tagged_words))

55304


In [79]:
# Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_given_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_given_w_with_given_tag = len(w_given_tag_list)
 
     
    return (count_given_w_with_given_tag, count_given_tag)

In [80]:
#  Transition Probability
def t2_given_t1(tag2, tag1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==tag1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==tag1 and tags[index+1] == tag2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [90]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        aux = t2_given_t1(t2, t1)
        tags_matrix[i, j] = aux[0]/aux[1]
 
#print(tags_matrix)

In [96]:
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,NUM,ADP,PART,NOUN,ADV,CCONJ,PUNCT,INTJ,SYM,DET,X,SCONJ,PRON,ADJ,VERB,AUX
NUM,0.019324,0.113527,0.004831,0.415459,0.096618,0.060386,0.024155,0.004831,0.000000,0.009662,0.002415,0.016908,0.033816,0.103865,0.067633,0.026570
ADP,0.016052,0.063912,0.004013,0.481570,0.049792,0.003864,0.006094,0.000446,0.000000,0.063317,0.000743,0.007134,0.089923,0.098246,0.066142,0.048751
PART,0.004071,0.062415,0.010855,0.169607,0.075984,0.039349,0.023066,0.000000,0.001357,0.021710,0.000000,0.012212,0.149254,0.062415,0.151967,0.215739
NOUN,0.005457,0.172964,0.011536,0.197209,0.101886,0.067279,0.066796,0.002832,0.000138,0.036541,0.001174,0.020861,0.098777,0.058576,0.082959,0.075016
ADV,0.006150,0.090855,0.018052,0.234279,0.086491,0.023805,0.034914,0.000397,0.000397,0.030946,0.000793,0.022416,0.120413,0.065265,0.131522,0.133307
CCONJ,0.007356,0.081839,0.024368,0.218851,0.092414,0.008276,0.060230,0.000460,0.000460,0.026207,0.008736,0.015632,0.119540,0.063908,0.172414,0.099310
PUNCT,0.011603,0.087667,0.029222,0.219596,0.137086,0.050709,0.012462,0.005157,0.000859,0.049850,0.001289,0.045122,0.120756,0.042544,0.081220,0.104856
INTJ,0.000000,0.080357,0.008929,0.321429,0.169643,0.017857,0.053571,0.000000,0.000000,0.044643,0.000000,0.008929,0.116071,0.071429,0.044643,0.062500
SYM,0.000000,0.125000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.375000,0.250000,0.187500
DET,0.007554,0.065156,0.011331,0.406043,0.077904,0.031161,0.034466,0.002361,0.000000,0.021719,0.000472,0.011331,0.088291,0.113787,0.063267,0.065156


In [92]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['PUNCT', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [95]:
test_la_misto = "Sănt îndoauă învețuri, cari si amintă mași cu ghivăsirea, i cu avdzărea."
test_la_misto = test_la_misto.split()

testt = Viterbi(test_la_misto)

print(testt)

[('Sănt', 'NOUN'), ('îndoauă', 'DET'), ('învețuri,', 'NOUN'), ('cari', 'PRON'), ('si', 'PRON'), ('amintă', 'VERB'), ('mași', 'ADV'), ('cu', 'ADP'), ('ghivăsirea,', 'NOUN'), ('i', 'CCONJ'), ('cu', 'ADP'), ('avdzărea.', 'NOUN')]
